In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR='/content/drive/MyDrive/BIR_Workshop/model_mesh'

In [4]:
!pip install livelossplot --quiet

In [5]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torchtext
from torch.utils.data import Dataset,DataLoader
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
from livelossplot import PlotLosses


device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')

NUM_CLASSES = 195
BATCH_SIZE = 128

MeSH Dataset Class...

In [6]:
class MESHDataset(Dataset):
    def __init__(self,numpy_file,label_file):
     
      try:
        self.data= np.load(numpy_file)
        self.labels = np.load(label_file)
      except Exception as err:
        raise Exception(f'ERROR OPENING FILES: {numpy_file} | {label_file}. See Error below. \n {err}')  
       
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        #Get the element with `idx`
        #Output an 89*89 matrix
        return self.data[idx].flatten(), self.labels[idx]
      

Baseline Model...

In [7]:
class BaselineModel(nn.Module):

    def __init__(self, matrix_size=89):
        super(BaselineModel, self).__init__()
        self.linear1 = nn.Linear(matrix_size**2, (matrix_size**2)//2)
        self.linear2 = nn.Linear((matrix_size**2)//2, matrix_size**2//4)
        self.linear3 = nn.Linear((matrix_size**2)//4, NUM_CLASSES)
        
    def forward(self, x):

        x = self.linear1(x)
        x = F.relu(x) 

        x = self.linear2(x)
        x = F.relu(x)

        x = self.linear3(x)
        return x

Creating the dataloaders...

In [8]:
# train_dataset = MESHDataset(os.path.join(DATA_DIR,'train.npy'), os.path.join(DATA_DIR,'train_labels.npy'))
# dev_dataset = MESHDataset(os.path.join(DATA_DIR,'dev.npy'), os.path.join(DATA_DIR,'dev_labels.npy'))
# test_dataset = MESHDataset(os.path.join(DATA_DIR,'test.npy'),os.path.join(DATA_DIR,'test_labels.npy'))

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Creating the model...


In [9]:
model = BaselineModel().to(device)
model

BaselineModel(
  (linear1): Linear(in_features=7921, out_features=3960, bias=True)
  (linear2): Linear(in_features=3960, out_features=1980, bias=True)
  (linear3): Linear(in_features=1980, out_features=5, bias=True)
)

Training Configuration...

In [10]:
from sklearn.metrics import accuracy_score
def compute_accuracy(pred, target):
  return float(100 * accuracy_score(target.detach().numpy(), pred.argmax(-1).detach().numpy()))

In [ ]:
#Define criterion: Categorical cross entropy
criterion = nn.CrossEntropyLoss()
#Define optimizer. For now, Adam
optimizer = optim.Adam(model.parameters(), lr=0.0001)

PRINT_FREQ = 15
DO_VALIDATION_STEP=15
NUM_EPOCHS = 1500
PATIENCE=50

PATH=os.path.join(DATA_DIR,'best_model_1')
best_acc = 0

liveloss = PlotLosses()
patience=0
for epoch in range(NUM_EPOCHS):
  for i,(features, label) in enumerate(train_dataloader):
    features, label = features.to(device), label.to(device)
    optimizer.zero_grad()
    pred = model(features.float())
    loss = criterion(pred,label)
    
    loss.backward()
    optimizer.step()

    if i%DO_VALIDATION_STEP==0:
      #Do validation
      model.eval()
      val_losses=[]
      val_acc_list=[]
      for val_features, val_label in dev_dataloader:
         val_features, val_label = val_features.to(device), val_label.to(device)
         val_pred = model(val_features.float())
         val_loss = criterion(val_pred,val_label)
         val_losses.append(val_loss.item())
         val_acc_list.append(compute_accuracy(val_pred.cpu(),val_label.cpu()))

      val_loss_ = np.mean(val_losses) 
      val_acc = np.mean(val_acc_list)
      if val_acc> best_acc:
        best_acc=val_acc
        patience=0
        #Save model weights
        torch.save(model.state_dict(), PATH)
      else:
        patience+=1
      if patience >= PATIENCE:
        break
      logs={'loss':loss.item(),'val_loss':val_loss_,'val_accuracy':val_acc}
      liveloss.update(logs)
      liveloss.send()  
      model.train()

Compute test metrics... 

In [ ]:
model.eval()
test_acc_list=[]
#f1_list=[]
for test_features, test_label in test_dataloader:
  test_features, test_label =test_features.to(device), test_label.to(device)
  test_pred = model(test_features.float())

  test_acc_list.append(compute_accuracy(test_pred.cpu(),test_label.cpu()))
  #f1_list.append(f1_loss(test_label, test_pred).item())

test_acc = np.mean(test_acc_list)
#test_f1 = np.mean(f1_list)

print(f'Test Metrics \n _________________ \n Mean Accuracy: {test_acc} ')     